In [105]:
# !pip -q install pycbc

You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [106]:
import os
import json
import random
import collections

import numpy as np
import pandas as pd
import torch



import pycbc
import pycbc.psd
import pylab
from pycbc.psd import welch, interpolate

from scipy import signal
# from scipy.fft import fft, fftshift

import joblib
from tqdm.auto import tqdm
import shutil
import matplotlib.pyplot as plt
import torch


In [107]:
class Config:

    input_data_folder = 'data/'
    input_train = input_data_folder + 'train/'
    input_test = input_data_folder + 'test/'
    output_folder = './data/'
    output_folder_train = output_folder + 'whiten-train-w0/'
    output_folder_test = output_folder + 'whiten-test-w0/'
    
    debugging_mode = False
    whiten_train = True#generate whitened training data or not
    whiten_test= True#generate whitened testing data or not


    #whiten params
    window = 'tukey'
    alpha = 0.5
    # window = 'planck'
    # alpha = 0.25
#     window = 'hann'
#     alpha = None
#     window = 'nuttall'
#     alpha = None
    #other
    fp16 = False
np.set_printoptions(precision=7, suppress=False) 

In [108]:
if not os.path.isdir(Config.input_train):
    !mkdir -p $Config.input_train
if not os.path.isdir(Config.input_test):
    !mkdir -p $Config.input_test
if not os.path.isdir(Config.output_folder_train):
    !mkdir -p $Config.output_folder_train
if not os.path.isdir(Config.output_folder_test):
    !mkdir -p $Config.output_folder_test

In [109]:
def get_train_file_path(image_id):
    return "{}train/{}/{}/{}/{}.npy".format(Config.input_data_folder,
        image_id[0], image_id[1], image_id[2], image_id)
def get_test_file_path(image_id):
    return "{}test/{}/{}/{}/{}.npy".format(Config.input_data_folder,
        image_id[0], image_id[1], image_id[2], image_id)
if Config.whiten_train:
    train = pd.read_csv(f'training_labels.csv')
    train['file_path'] = train['id'].apply(get_train_file_path)
    display(train.head())
if Config.whiten_test:
    test = pd.read_csv(f'sample_submission.csv')
    test['file_path'] = test['id'].apply(get_test_file_path)
    display(test.head())

,id,target,file_path
0,00000e74ad,1,train/0/0/0/00000e74ad.npy
1,00001f4945,0,train/0/0/0/00001f4945.npy
2,0000661522,0,train/0/0/0/0000661522.npy
3,00007a006a,0,train/0/0/0/00007a006a.npy
4,0000a38978,1,train/0/0/0/0000a38978.npy


In [110]:
if Config.debugging_mode:
    if Config.whiten_train:
        train = train[:5000]
        print(len(train))
    if Config.whiten_test:
        test = test[:5000]
        print(len(test))

In [ ]:
#for planck window

In [133]:
"""The suite of window functions."""

import operator
import warnings

import numpy as np
from scipy import linalg, special, fft as sp_fft

__all__ = ['boxcar', 'triang', 'parzen', 'bohman', 'blackman', 'nuttall',
           'blackmanharris', 'flattop', 'bartlett', 'hanning', 'barthann',
           'hamming', 'kaiser', 'gaussian', 'general_cosine',
           'general_gaussian', 'general_hamming', 'chebwin', 'cosine',
           'hann', 'exponential', 'tukey', 'taylor', 'dpss', 'get_window']


def _len_guards(M):
    """Handle small or incorrect window lengths"""
    if int(M) != M or M < 0:
        raise ValueError('Window length M must be a non-negative integer')
    return M <= 1


def _extend(M, sym):
    """Extend window by 1 sample if needed for DFT-even symmetry"""
    if not sym:
        return M + 1, True
    else:
        return M, False


def _truncate(w, needed):
    """Truncate window by 1 sample if needed for DFT-even symmetry"""
    if needed:
        return w[:-1]
    else:
        return w

def planck(M, alpha=0.5, sym=True):
    r"""Return a Tukey window, also known as a tapered cosine window.
    Parameters
    ----------
    M : int
        Number of points in the output window. If zero or less, an empty
        array is returned.
    alpha : float, optional
        Shape parameter of the Tukey window, representing the fraction of the
        window inside the cosine tapered region.
        If zero, the Tukey window is equivalent to a rectangular window.
        If one, the Tukey window is equivalent to a Hann window.
    sym : bool, optional
        When True (default), generates a symmetric window, for use in filter
        design.
        When False, generates a periodic window, for use in spectral analysis.
    Returns
    -------
    w : ndarray
        The window, with the maximum value normalized to 1 (though the value 1
        does not appear if `M` is even and `sym` is True).
    References
    ----------
    .. [1] Harris, Fredric J. (Jan 1978). "On the use of Windows for Harmonic
           Analysis with the Discrete Fourier Transform". Proceedings of the
           IEEE 66 (1): 51-83. :doi:`10.1109/PROC.1978.10837`
    .. [2] Wikipedia, "Window function",
           https://en.wikipedia.org/wiki/Window_function#Tukey_window
    Examples
    --------
    Plot the window and its frequency response:
    >>> from scipy import signal
    >>> from scipy.fft import fft, fftshift
    >>> import matplotlib.pyplot as plt
    >>> window = signal.windows.tukey(51)
    >>> plt.plot(window)
    >>> plt.title("Tukey window")
    >>> plt.ylabel("Amplitude")
    >>> plt.xlabel("Sample")
    >>> plt.ylim([0, 1.1])
    >>> plt.figure()
    >>> A = fft(window, 2048) / (len(window)/2.0)
    >>> freq = np.linspace(-0.5, 0.5, len(A))
    >>> response = 20 * np.log10(np.abs(fftshift(A / abs(A).max())))
    >>> plt.plot(freq, response)
    >>> plt.axis([-0.5, 0.5, -120, 0])
    >>> plt.title("Frequency response of the Tukey window")
    >>> plt.ylabel("Normalized magnitude [dB]")
    >>> plt.xlabel("Normalized frequency [cycles per sample]")
    """
    if _len_guards(M):
        return np.ones(M)

    if alpha <= 0:
        return np.ones(M, 'd')
    elif alpha >= 1.0:
        return hann(M, sym=sym)

    M, needs_trunc = _extend(M, sym)

    n = np.arange(0, M)
    width = int(np.floor(alpha*(M-1)))
    n1 = n[1:width+1]
    n2 = n[width+1:M-width-1]
    n3 = n[M-width-1:-1]
    N = M - 1
    w1 = 1/(1+np.exp(alpha*N/n1-alpha*N/(alpha*N-n1)))#0.5 * (1 + np.cos(np.pi * (-1 + 2.0*n1/alpha/(M-1))))
    w2 = np.ones(n2.shape)
    w3 = 1/(1+np.exp(alpha*N/(N-n3)-alpha*N/(alpha*N-N+n3)))#0.5 * (1 + np.cos(np.pi * (-2.0/alpha + 1 + 2.0*n3/alpha/(M-1))))

    w = np.concatenate((np.array([0]),w1, w2, w3, np.array([0])))

    return _truncate(w, needs_trunc)

In [134]:
if Config.window=='tukey':
    window = signal.windows.tukey(4096+2*2048,Config.alpha)
elif Config.window=='planck':
    window = planck(4096+2*2048,Config.alpha)
elif Config.window=='hann':
    window = signal.windows.hann(4096+2*2048)
elif Config.window=='nuttall':
    window = signal.windows.nuttall(4096+2*2048)

In [135]:
window

array([0.0000000e+00, 1.4710447e-07, 5.8841780e-07, ..., 5.8841780e-07,
       1.4710447e-07, 0.0000000e+00])

In [136]:
len(window)

8192

In [137]:
# avr_w_all = torch.load("./avr_w_all.pth")
# avr_w = torch.load("./avr_w.pth")
avr_w0 = torch.load("./avr_w0.pth")
window = torch.FloatTensor(window)
def process_wave(waves, window, avr_w):
    c = torch.FloatTensor(waves)
    c = torch.cat([-c.flip(-1)[:,4096-2049:-1]+2*c[:,0].unsqueeze(-1),c,
                                      -c.flip(-1)[:,1:2049]+2*c[:,-1].unsqueeze(-1)],1)    
    c = torch.fft.ifft(torch.fft.fft((1e20*c*window))/avr_w0).real[:,2048:-2048]
    return c.numpy()
# waves = np.load(train.iloc[0,-1])



In [138]:
def save_processed_waves(input_file_path, output_folder, avg_asd=None):
    file_name = input_file_path.split('/')[-1].split('.npy')[0]
    waves = np.load(input_file_path).astype(np.float32) # (3, 4096)
    processed_waves = process_wave(waves, window, avr_w0).astype(np.float32)
    np.save(output_folder + file_name, processed_waves)

In [139]:
if Config.whiten_train:
    _ = joblib.Parallel(n_jobs=8)(
        joblib.delayed(save_processed_waves)(input_file_path,Config.output_folder_train) for input_file_path in tqdm(train['file_path'].values)
    )
if Config.whiten_test:
    _ = joblib.Parallel(n_jobs=8)(
        joblib.delayed(save_processed_waves)(input_file_path,Config.output_folder_test) for input_file_path in tqdm(test['file_path'].values)
    )